# Stellar Migration

In [1]:
import os
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
from scipy.stats import gaussian_kde
import yaml
from multiprocessing import Pool

In [3]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup
from auriga.settings import Settings
from auriga.support import find_indices
from auriga.parser import parse

In [4]:
figure_setup()

In [5]:
settings = Settings()

In [6]:
CONFIG_FILE = "02"
config = yaml.safe_load(open(f"../configs/{CONFIG_FILE}.yml"))

## Calculate Stellar Migration

In [7]:
def read_data(simulation: str, config: dict) -> pd.DataFrame:
    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.tag_in_situ_stars()
    s.tag_particles_by_region(
        disc_std_circ=config["DISC_STD_CIRC"],
        disc_min_circ=config["DISC_MIN_CIRC"],
        cold_disc_delta_circ=config["COLD_DISC_DELTA_CIRC"],
        bulge_max_specific_energy=config["BULGE_MAX_SPECIFIC_ENERGY"])
    s.add_stellar_age()

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)
    is_target = is_real_star & is_main_obj

    props = {
        "ID": s.ids[is_target],
        "ComponentTag": s.region_tag[is_target],
        "Mass_Msun": s.mass[is_target],
        "IsInSitu": s.is_in_situ[is_target],
        "FormationSnapshot": s.stellar_formation_snapshot[is_target],
        "StellarAge_Gyr": s.stellar_age[is_target],
        }
    
    # region Abundances
    for of, to in [("Fe", "H"), ("O", "H"), ("O", "Fe")]:
        s.add_metal_abundance(of, to)
        props[f"Abundance_[{of}/{to}]"] = s.metal_abundance[
            f"{of}/{to}"][is_target]
    # endregion

    return pd.DataFrame(props)

In [8]:
def read_reduced_data(simulation: str, config: dict) -> pd.DataFrame:
    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.tag_particles_by_region(
        disc_std_circ=config["DISC_STD_CIRC"],
        disc_min_circ=config["DISC_MIN_CIRC"],
        cold_disc_delta_circ=config["COLD_DISC_DELTA_CIRC"],
        bulge_max_specific_energy=config["BULGE_MAX_SPECIFIC_ENERGY"])

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)
    is_target = is_real_star & is_main_obj

    props = {
        "ID": s.ids[is_target],
        "Mass_Msun": s.mass[is_target],
        "ComponentTag": s.region_tag[is_target],
        }

    return pd.DataFrame(props)

In [9]:
def calculate_migration_matrix(simulation: str, config: dict) -> None:
    settings = Settings()
    df = read_data(f"{simulation}_s127", config)

    # region Find Component at Birth
    component_at_birth = -1 * np.ones(len(df), dtype=np.int8)
    for i in range(40, 127, 1):
        this_df = read_reduced_data(f"{simulation}_s{i}", config)

        idx = find_indices(
            this_df["ID"].to_numpy(),
            df["ID"][
                (df["FormationSnapshot"] == i) & df["IsInSitu"]].to_numpy(),
            -1)
        idx = idx[idx >= 0]

        component_at_birth[(df["FormationSnapshot"] == i) & df["IsInSitu"]] \
            = this_df["ComponentTag"][idx].to_numpy()

    # Add component tags at birth for the last snapshot
    component_at_birth[(df["FormationSnapshot"] == 127) & df["IsInSitu"]] \
            = df["ComponentTag"][df["FormationSnapshot"] == 127].to_numpy()

    df["ComponentTagAtBirth"] = component_at_birth
    # endregion

    # region Migration Matrix
    migration_matrix = np.zeros((4, 4))
    for i in range(0, 4):
        mass = df["Mass_Msun"][
            (df["ComponentTag"] == i) & df["IsInSitu"]].sum()
        for j in range(0, 4):
            this_mass = df["Mass_Msun"][
                (df["ComponentTag"] == i) & df["IsInSitu"] \
                    & (df["ComponentTagAtBirth"] == j)].sum()
            migration_matrix[i, j] = this_mass / mass * 100
    np.save(f"../results/{simulation}/"
            f"migration_matrix{config['FILE_SUFFIX']}.npy", migration_matrix)
    # endregion

    # region Distribution of Stellar Ages
    age_distribution = {}
    pdf_x = np.linspace(0.0, 14.0, 100)
    for i in range(0, 4):
        c_to = settings.components[i]
        component_mass = df["Mass_Msun"][df["ComponentTag"] == i].sum()
        age_distribution[f"StellarMassFraction_{i}"] = np.histogram(
            a=df["StellarAge_Gyr"][df["ComponentTag"] == i],
            bins=50, range=(0, 14),
            weights=df["Mass_Msun"][df["ComponentTag"] == i] / component_mass)[0]
        age_distribution[f"StellarMassFraction_{i}_InSitu"] = np.histogram(
                a=df["StellarAge_Gyr"][
                    (df["IsInSitu"] == 1) & (df["ComponentTag"] == i)],
                bins=50, range=(0, 14),
                weights=df["Mass_Msun"][
                    (df["IsInSitu"] == 1) & (df["ComponentTag"] == i)
                    ] / component_mass)[0]
        age_distribution[
            f"StellarMassFraction_{i}_ExSitu"] = np.histogram(
                a=df["StellarAge_Gyr"][
                    (df["IsInSitu"] == 0) & (df["ComponentTag"] == i)],
                bins=50, range=(0, 14),
                weights=df["Mass_Msun"][
                    (df["IsInSitu"] == 0) & (df["ComponentTag"] == i)
                    ] / component_mass)[0]
        for j in range(0, 4):
            c_from = settings.components[j]
            mask = (df["ComponentTagAtBirth"] == j) \
                & df["IsInSitu"] & (df["ComponentTag"] == i)
            h, bin_edges = np.histogram(
                    a=df[f"StellarAge_Gyr"][mask],
                    bins=50, range=(0, 14),
                    weights=df[f"Mass_Msun"][mask] / component_mass,
                )
            bin_centers = bin_edges[1:] - np.diff(bin_edges) / 2
            age_distribution[f"StellarMassFraction_{j}to{i}"] = h
    age_distribution["StellarAge_Gyr"] = bin_centers
    age_distribution = pd.DataFrame(age_distribution)
    age_distribution.to_csv(
        f"../results/{simulation}/"
        f"stellar_migration_age_distribution{config['FILE_SUFFIX']}.csv")
    # endregion
    
    # region Distribution of Abundances
    abundances = [("Fe", "H"), ("O", "H"), ("O", "Fe")]
    abundance_ranges = [(-2.5, 1), (-2, 1.5), (0.1, 0.4)]
    for a, (of, to) in enumerate(abundances):
        abundance_distribution = {}
        for i in range(0, 4):
            c_to = settings.components[i]
            component_mass = df["Mass_Msun"][df["ComponentTag"] == i].sum()
            abundance_distribution[f"StellarMassFraction_{i}"] = np.histogram(
                a=df[f"Abundance_[{of}/{to}]"][df["ComponentTag"] == i],
                bins=50, range=abundance_ranges[a],
                weights=df["Mass_Msun"][df["ComponentTag"] == i] / component_mass)[0]
            abundance_distribution[
                f"StellarMassFraction_{i}_InSitu"] = np.histogram(
                    a=df[f"Abundance_[{of}/{to}]"][
                        (df["IsInSitu"] == 1) & (df["ComponentTag"] == i)],
                    bins=50, range=abundance_ranges[a],
                    weights=df["Mass_Msun"][
                        (df["IsInSitu"] == 1) & (df["ComponentTag"] == i)
                        ] / component_mass)[0]
            abundance_distribution[
                f"StellarMassFraction_{i}_ExSitu"] = np.histogram(
                    a=df[f"Abundance_[{of}/{to}]"][
                        (df["IsInSitu"] == 0) & (df["ComponentTag"] == i)],
                    bins=50, range=abundance_ranges[a],
                    weights=df["Mass_Msun"][
                        (df["IsInSitu"] == 0) & (df["ComponentTag"] == i)
                        ] / component_mass)[0]
            for j in range(0, 4):
                c_from = settings.components[j]
                mask = (df["ComponentTagAtBirth"] == j) \
                    & df["IsInSitu"] & (df["ComponentTag"] == i)
                h, bin_edges = np.histogram(
                    a=df[f"Abundance_[{of}/{to}]"][mask],
                    bins=50, range=abundance_ranges[a],
                    weights=df["Mass_Msun"][mask] / component_mass,
                )
                bin_centers = bin_edges[1:] - np.diff(bin_edges) / 2
                abundance_distribution[f"StellarMassFraction_{j}to{i}"] = h
        abundance_distribution[f"Abundance_[{of}/{to}]"] = bin_centers
        abundance_distribution = pd.DataFrame(abundance_distribution)
        abundance_distribution.to_csv(
            f"../results/{simulation}/"
            f"stellar_migration_abundance_{of}{to}"
            f"_dist{config['FILE_SUFFIX']}.csv")
    # endregion

    print(f"{simulation} Ok.")

In [11]:
# args = [(f"au{i}_or_l4", config) for i in settings.groups["Included"]]
args = [(f"au{i}_or_l4", config) for i in [24, 26, 27]]
_ = Pool(8).starmap(calculate_migration_matrix, args)

au27_or_l4 Ok.
au24_or_l4 Ok.
au26_or_l4 Ok.
